<a href="https://colab.research.google.com/github/Many98/real_estate/blob/criminality/notebooks/criminality_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import datetime 

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
pd.set_option('display.max_columns', None)
df_nopred = pd.read_csv('/content/drive/MyDrive/reality/dataset_2.csv', sep=',', delimiter=None, encoding="utf8")

In [ ]:
df_nopred.columns

Index(['city_district', 'additional_disposition', 'air_quality', 'atm_dist',
       'built_density', 'bus_station_dist', 'construction_type', 'daily_noise',
       'date', 'desc_hash', 'description', 'disposition', 'doctor_dist',
       'electricity', 'electricity_txt', 'energy_effeciency', 'equipment',
       'floor', 'floor_area', 'gas', 'geometry', 'gp_ci_high_price',
       'gp_ci_low_price', 'gp_mean_price', 'gp_std_price', 'has_balcony',
       'has_cellar', 'has_garage', 'has_garden', 'has_lift', 'has_loggia',
       'has_parking', 'hash', 'header', 'heating', 'heating_txt',
       'kindergarten_dist', 'lat', 'long', 'name', 'nightly_noise',
       'no_barriers', 'note', 'ownership', 'pharmacy_dist', 'place',
       'playground_dist', 'post_office_dist', 'price', 'primary_school_dist',
       'restaurant_pub_dist', 'sports_field_dist', 'state',
       'subway_station_dist', 'sun_glare', 'supermarket_grocery_dist', 'tags',
       'telecomunication', 'telecomunication_txt', 'theat

## Loading prepared dataset

In [86]:
df2 = pd.read_csv('/content/drive/MyDrive/reality/discounted_crime.csv', sep=',', delimiter=None, encoding="utf8")

In [87]:
df2.head()

,Unnamed: 0,index,id,x,y,date,types,crime_idx,cut_x,cut_y,disc_index
0,0,58,7917760,14.406693,50.068626,2017-01-02,vloupání do bytu,9.0,"(14.405, 14.407]","(50.0686, 50.0695]",5.31441
1,1,59,7917762,14.511259,49.995567,2017-01-02,vloupání do rodinných domů,9.0,"(14.51, 14.511]","(49.995, 49.9959]",5.31441
2,2,60,7917787,14.437774,50.005458,2017-01-02,krádeže součástek aut,4.0,"(14.437, 14.439]","(50.0049, 50.0058]",2.36196
3,3,61,7917799,14.441282,50.125784,2017-01-02,krádeže součástek aut,4.0,"(14.44, 14.442]","(50.1251, 50.126]",2.36196
4,4,62,7917812,14.305693,50.072810,2017-01-02,krádeže součástek aut,4.0,"(14.306, 14.307]","(50.0722, 50.0731]",2.36196


In [88]:
df3 = df2[['id', 'x',	'y', 'disc_index']].copy()

In [89]:
df3['xy'] = list(zip(df3.x, df3.y))

In [90]:
df3.head()

,id,x,y,disc_index,xy
0,7917760,14.406693,50.068626,5.31441,"(14.406693, 50.068626)"
1,7917762,14.511259,49.995567,5.31441,"(14.511259, 49.995567)"
2,7917787,14.437774,50.005458,2.36196,"(14.437774, 50.005458)"
3,7917799,14.441282,50.125784,2.36196,"(14.441282, 50.125784)"
4,7917812,14.305693,50.072810,2.36196,"(14.305693, 50.07281)"


In [91]:
df3 = df3[df3.columns.difference(['x', 'y'])]

In [92]:
help1 = df3.groupby(['xy']).apply(lambda df3: df3['disc_index'].sum())

In [93]:
df4 = df3.groupby( ['xy'] ).count()

In [94]:
df4['disc_index'] = help1

In [95]:
df_final = df4[df4.columns.difference(['id'])]

In [96]:
df_final.head()

,disc_index
xy,
"(14.252392, 50.059311)",7.41600
"(14.263671, 50.109646)",659.47632
"(14.268226, 50.10447)",615.69369
"(14.26839, 50.059282)",44.07412
"(14.26859, 50.096719)",12.53880


Maybe to separate x-y?

In [99]:
df_final= df_final.reset_index()

In [101]:
df_final['xy'].nunique()

1714

In [102]:
df_final[['x', 'y']] = pd.DataFrame(df_final['xy'].tolist(), index=df_final.index)

In [103]:
df_final['x'].nunique()

1711

In [104]:
df_final['y'].nunique()

1704

In [105]:
result_df = df_final.drop_duplicates(subset=['x', 'y'], keep='first')

In [106]:
result_df['x'].nunique()

1711

In [121]:
df_final = df_final.drop_duplicates(subset=['y'], keep='first')
df_final = df_final.drop_duplicates(subset=['x'], keep='first')

In [123]:
df_final = df_final[df_final.columns.difference(['xy'])]

In [122]:
df_final.head()

,xy,disc_index,x,y
0,"(14.252392, 50.059311)",7.41600,14.252392,50.059311
1,"(14.263671, 50.109646)",659.47632,14.263671,50.109646
2,"(14.268226, 50.10447)",615.69369,14.268226,50.104470
3,"(14.26839, 50.059282)",44.07412,14.268390,50.059282
4,"(14.26859, 50.096719)",12.53880,14.268590,50.096719


In [124]:
df_final = df_final.set_index(['x', 'y'])

In [125]:
df_final['disc_index'].isna().sum()

0

In [126]:
crim_dataset = xr.Dataset.from_dataframe(df_final)

In [127]:
crim_dataset

<xarray.Dataset>
Dimensions:     (x: 1701, y: 1701)
Coordinates:
  * x           (x) float64 14.25 14.26 14.27 14.27 ... 14.68 14.68 14.69 14.7
  * y           (y) float64 49.95 49.95 49.95 49.95 ... 50.17 50.17 50.17 50.17
Data variables:
    disc_index  (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan

In [130]:
df_final= df_final.reset_index()
df_final.head()

,index,x,y,disc_index
0,0,14.252392,50.059311,7.41600
1,1,14.263671,50.109646,659.47632
2,2,14.268226,50.104470,615.69369
3,3,14.268390,50.059282,44.07412
4,4,14.268590,50.096719,12.53880


In [131]:
crim_dataset2 = xr.Dataset(
    data_vars=dict(
        disc_index=(["x", "y"], df_final['disc_index']),
    ),
    coords=dict(
        lon=(["x", "y"], df_final['x']),
        lat=(["x", "y"], df_final['y']),
    ),
    attrs=dict(description="Prague crimes map data."),
)

ValueError: ignored

Dalsi pokusy

## Data v polickach

In [ ]:
df3.groupby(['cut_x', 'cut_y']).apply(lambda df3: df3['disc_index'].sum())

cut_x             cut_y             
(14.252, 14.254]  (50.0587, 50.0596]      7.41600
(14.263, 14.265]  (50.109, 50.1099]     659.47632
(14.267, 14.268]  (50.1036, 50.1045]    615.69369
(14.268, 14.27]   (50.0587, 50.0596]     44.07412
                  (50.0964, 50.0973]     12.53880
                                          ...    
(14.68, 14.682]   (50.0722, 50.0731]    151.45802
                  (50.0767, 50.0776]     67.72086
(14.684, 14.685]  (50.0964, 50.0973]     18.47700
(14.687, 14.689]  (50.0731, 50.074]      83.96901
(14.694, 14.696]  (50.0865, 50.0874]      6.56100
Length: 1714, dtype: float64

In [ ]:
type(df3.loc[0,'cut_x'])

str

In [ ]:
import ast

In [ ]:
def interval_type(s):
    """Parse interval string to Interval"""
    
    table = str.maketrans({'[': '(', ']': ')'})
    left_closed = s.startswith('[')
    right_closed = s.endswith(']')

    left, right = ast.literal_eval(s.translate(table))

    t = 'neither'
    if left_closed and right_closed:
        t = 'both'
    elif left_closed:
        t = 'left'
    elif right_closed:
        t = 'right'

    return pd.Interval(left, right, closed=t)

In [ ]:
df3['cut_x'] = df3['cut_x'].apply(interval_type)
df3['cut_y'] = df3['cut_y'].apply(interval_type)

In [ ]:
d = {'cut_x': np.zeros(len(df3['cut_x'])), 'cut_y': np.zeros(len(df3['cut_x'])), 'disc_index': np.zeros(len(df3['cut_x']))}
df4 = pd.DataFrame(data=d)
for x in range(len(df3['cut_x'])):
  df4.loc[x,'cut_x'] = df3.loc[x, 'cut_x'].left
  df4.loc[x,'cut_y'] = df3.loc[x,'cut_y'].left
  df4.loc[x,'disc_index'] = df3.loc[x, 'disc_index']

In [ ]:
df4.head()

,cut_x,cut_y,disc_index
0,14.405,50.0686,5.31441
1,14.510,49.9950,5.31441
2,14.437,50.0049,2.36196
3,14.440,50.1251,2.36196
4,14.306,50.0722,2.36196


In [ ]:
y_max, y_min = np.max(df4['cut_y']), np.min(df4['cut_y'])
x_max, x_min = np.max(df4['cut_x']), np.min(df4['cut_x'])
x_co = np.linspace(x_min, x_max, 250)
y_co = np.linspace(y_min, y_max, 250)

In [ ]:
x_coords2 = xr.DataArray(x_co, dims="points")
y_coords2 = xr.DataArray(y_co, dims="points")

In [ ]:
xy_coo = xr.DataArray(np.zeros((250,250)),[("x", x_co), ("y", y_co)]) #, ("y", y_co)

In [ ]:
df_multiindex = df4.set_index(['cut_x', 'cut_y'])

In [ ]:
df_multiindex.to_xarray()

ValueError: ignored

In [ ]:
len(df4['cut_y'].unique())

227

In [ ]:
len(df4['cut_x'].unique())

230

In [ ]:
len(df4['cut_x'])

115087

## Also different approach

In [132]:
y_max, y_min = np.max(df_nopred['lat']), np.min(df_nopred['lat'])
x_max, x_min = np.max(df_nopred['long']), np.min(df_nopred['long'])
x_co = np.linspace(x_min, x_max, 250)
y_co = np.linspace(y_min, y_max, 250)

In [133]:
xy_coo = xr.DataArray(np.zeros((250,250)),[("x", x_co), ("y", y_co)]) #, ("y", y_co)

In [134]:
xy_coo

<xarray.DataArray (x: 250, y: 250)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * x        (x) float64 13.99 13.99 13.99 14.0 14.0 ... 14.67 14.68 14.68 14.68
  * y        (y) float64 49.9 49.9 49.9 49.9 49.9 ... 50.16 50.16 50.16 50.16